In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pandas as pd # data processing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv1D,Dropout,LSTM, Bidirectional
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from keras.layers import BatchNormalization, Dropout
from keras.layers import Dense, Activation
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import torch
from torch.nn import Transformer
from sklearn.preprocessing import MinMaxScaler

In [2]:
FS = 30 # sampling rate
TOTAL_CLASSESS = 9
DATA_FOLDER = "./"
WINDOW_SIZE = 2 # seconds
OVERLAP_RATE = 0.25 * WINDOW_SIZE # overlap 50% of window size
standartscaler=MinMaxScaler()
def load_data(keypoint_csv, ann_csv):
    kp_df = pd.read_csv(keypoint_csv)
    kp_df = kp_df.loc[:, ~kp_df.columns.str.contains('^Unnamed')]

    ann_df = pd.read_csv(ann_csv)
    ann_df = ann_df.loc[:, ~ann_df.columns.str.contains('^Unnamed')]

    return kp_df, ann_df

In [3]:
def remove_redundant_kp(kp_df):
    kp_df = kp_df.loc[:, ~kp_df.columns.str.contains(
    'conf|left_knee|right_knee|left_ankle|right_ankle', regex=True)]
    return kp_df


In [4]:
SMOOTH_LEN = 3 # seconds

def smooth_kp(kp_col):
    zero_idx = np.flatnonzero(kp_col == 0)
    split_idx = np.split(zero_idx, np.flatnonzero(np.diff(zero_idx) > 1) + 1)
    for each_split_idx in split_idx:
        if len(each_split_idx) == 0 or each_split_idx[0] == 0 or each_split_idx[-1] == (len(kp_col) - 1) or len(each_split_idx) > SMOOTH_LEN*FS:
            continue
        xp = [each_split_idx[0] - 1, each_split_idx[-1] + 1]
        fp = kp_col[xp]
        interp_kp = np.interp(each_split_idx, xp, fp)
        kp_col[each_split_idx] = interp_kp
    return kp_col

In [5]:
def cal_angle(a, b, c):
    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return angle


def extract_joint_angles(kp_data, steps=2):
    # steps = 2 if kp_data is removed conf columns
    # steps = 3 if kp_data has conf columns
    left_elbow_shoulder_hip = np.asarray([cal_angle(kp_data[i, 7*steps:(7*steps+2)], kp_data[i, 5*steps:(5*steps+2)], kp_data[i, 11*steps:(11*steps+2)])
                                          for i in range(len(kp_data))])
    left_elbow_shoulder_hip = np.nan_to_num(left_elbow_shoulder_hip)


    right_elbow_shoulder_hip = np.asarray([cal_angle(kp_data[i, 8*steps:(8*steps+2)], kp_data[i, 6*steps:(6*steps+2)], kp_data[i, 12*steps:(12*steps+2)])
                                            for i in range(len(kp_data))])
    right_elbow_shoulder_hip = np.nan_to_num(right_elbow_shoulder_hip)


    left_wrist_elbow_shoulder = np.asarray([cal_angle(kp_data[i, 9*steps:(9*steps+2)], kp_data[i, 7*steps:(7*steps+2)], kp_data[i, 5*steps:(5*steps + 2)])
                                            for i in range(len(kp_data))])
    left_wrist_elbow_shoulder = np.nan_to_num(left_wrist_elbow_shoulder)


    right_wrist_elbow_shoulder = np.asarray([cal_angle(kp_data[i, 10*steps:(10*steps+2)], kp_data[i, 8*steps:(8*steps+2)], kp_data[i, 6*steps:(6*steps+2)])
                                              for i in range(len(kp_data))])
    right_wrist_elbow_shoulder = np.nan_to_num(right_wrist_elbow_shoulder)


    right_elbow_shoulder = np.asarray([cal_angle(kp_data[i, 8*steps:(8*steps+2)], kp_data[i, 6*steps:(6*steps+2)], kp_data[i, 5*steps:(5*steps+2)])
                                              for i in range(len(kp_data))])
    right_elbow_shoulder = np.nan_to_num(right_elbow_shoulder)


    left_elbow_shoulder = np.asarray([cal_angle(kp_data[i, 6*steps:(6*steps+2)], kp_data[i, 5*steps:(5*steps+2)], kp_data[i, 7*steps:(7*steps+2)])
                                              for i in range(len(kp_data))])
    left_elbow_shoulder = np.nan_to_num(left_elbow_shoulder)


    joint_angles = np.array([left_elbow_shoulder_hip,
                    right_elbow_shoulder_hip, left_wrist_elbow_shoulder, 
                             right_wrist_elbow_shoulder, right_elbow_shoulder,
                             left_elbow_shoulder]).T

    return joint_angles
def extract_velocity(kp_data):
    velocity = np.diff(kp_data, axis=0)
    return velocity
def segment(data, max_time, sub_window_size, stride_size):
    sub_windows = np.arange(sub_window_size)[None, :] + np.arange(0, max_time, stride_size)[:, None]

    row, col = np.where(sub_windows >= max_time)
    uniq_row = len(np.unique(row))

    if uniq_row > 0 and row[0] > 0:
        sub_windows = sub_windows[:-uniq_row, :]

    return data[sub_windows]
import scipy

def extract_feature(data, fs):
    mean_ft = np.mean(data, axis=0)
    std_ft = np.std(data, axis=0)
    max_ft = np.max(data, axis=0)
    min_ft = np.min(data, axis=0)
    var_ft = np.var(data, axis=0)
    med_ft = np.median(data, axis=0)
    sum_ft = np.sum(data, axis=0)
    std = np.std(data, axis=0)
    kurtosis=scipy.stats.kurtosis(data)
    skew=scipy.stats.skew(data)
    features = np.array([mean_ft, std_ft, max_ft, min_ft, var_ft, med_ft, sum_ft,std,kurtosis,skew]).T.flatten()
    features = np.nan_to_num(features)
    return features
def remove_redundant_kp(kp_df):
    kp_df = kp_df.loc[:, ~kp_df.columns.str.contains(
    'conf|left_knee|right_knee|left_ankle|right_ankle', regex=True)]
    return kp_df

In [8]:
def make_csv(USER_ID):
    USER_ID=USER_ID[:5]
    keypoint_csv = DATA_FOLDER + "train/{}_keypoint.csv".format(USER_ID)
    ann_csv = DATA_FOLDER + "/ann/{}_ann.csv".format(USER_ID)
    kp_df, ann_df = load_data(keypoint_csv, ann_csv)
    kp_df = remove_redundant_kp(kp_df)
    col=kp_df.columns
    kp_df=standartscaler.fit_transform(kp_df)
    kp_df=pd.DataFrame(kp_df,columns=col)
    feature_seg_feat=["left_elbow_shoulder_hip",
                    "right_elbow_shoulder_hip", "left_wrist_elbow_shoulder", 
                    "right_wrist_elbow_shoulder"," right_elbow_shoulder",
                    "left_elbow_shoulder"]
    norm_feat=['nose_x', 'nose_y', 'left_eye_x', 'left_eye_y', 'right_eye_x',
       'right_eye_y', 'left_ear_x', 'left_ear_y', 'right_ear_x', 'right_ear_y',
       'left_shoulder_x', 'left_shoulder_y', 'right_shoulder_x',
       'right_shoulder_y', 'left_elbow_x', 'left_elbow_y', 'right_elbow_x',
       'right_elbow_y', 'left_wrist_x', 'left_wrist_y', 'right_wrist_x',
       'right_wrist_y', 'left_hip_x', 'left_hip_y', 'right_hip_x',
       'right_hip_y', 'right_shoulder nose_ dx', 'right_shoulder nose dy',
       'right_shoulder left_eye_ dx', 'right_shoulder left_eye dy',
       'right_shoulder right_eye_ dx', 'right_shoulder right_eye dy',
       'right_shoulder left_ear_ dx', 'right_shoulder left_ear dy',
       'right_shoulder right_ear_ dx', 'right_shoulder right_ear dy',
       'right_shoulder left_shoulder_ dx', 'right_shoulder left_shoulder dy',
       'right_shoulder left_elbow_ dx', 'right_shoulder left_elbow dy',
       'right_shoulder right_elbow_ dx', 'right_shoulder right_elbow dy',
       'right_shoulder left_wrist_ dx', 'right_shoulder left_wrist dy',
       'right_shoulder right_wrist_ dx', 'right_shoulder right_wrist dy',
       'right_shoulder left_hip_ dx', 'right_shoulder left_hip dy',
       'right_shoulder right_hip_ dx', 'right_shoulder right_hip dy']
    vel_feat=[f"velo{x}" for x in norm_feat]
    feature=[]
    stat_feat=["mean_ft", "std_ft", "max_ft", "min_ft", "var_ft", "med_ft", "sum_ft","std","kurtosis","skew"]
    for feat in feature_seg_feat:
        for f in stat_feat:
            feature.append("".join([feat," ",f]))

    for feat in norm_feat:
        for f in stat_feat:
            feature.append("".join([feat," ",f]))

    for feat in vel_feat:
        for f in stat_feat:
            feature.append("".join([feat," ",f]))

    for i in [e for e in kp_df.columns if e not in ('right_shoulder_y','right_shoulder_x')]:
        if i[-1]=='y':
            feat_name="right_shoulder "+i[:-2]+' dy'
            kp_df[feat_name]=kp_df['right_shoulder_y']-kp_df[i]
        else:
            feat_name="right_shoulder "+i[:-1]+' dx'
            kp_df[feat_name]=kp_df['right_shoulder_x']-kp_df[i]
    
    for i in range(len(kp_df.columns)):
        kp_df.iloc[:, i] = smooth_kp(np.array(kp_df.iloc[:, i]))

    # Processing training data
    all_feature = []
    all_label = []

    for i in range(len(ann_df)):
        seg = kp_df.loc[int(ann_df['start_time'][i]*FS): int(ann_df['stop_time'][i]*FS)]
        seg_label = ann_df["annotation"].iloc[i]
        if len(seg) > 0 and (len(seg) >= WINDOW_SIZE * FS):
            # Calculate joint angles from keypoint data
            joint_angles = extract_joint_angles(np.array(seg))

            # Segment keypoint data and joint angles by WINDOW_SIZE and OVERLAP_RATE
            ws_seg = segment(np.array(seg), max_time=len(seg), sub_window_size=WINDOW_SIZE * FS, stride_size=int((WINDOW_SIZE - OVERLAP_RATE) * FS))
            joint_angles_seg = segment(joint_angles, max_time=len(seg), sub_window_size=WINDOW_SIZE * FS,
                                            stride_size=int((WINDOW_SIZE - OVERLAP_RATE) * FS))

            # Calculate velocity from each segment of keypoint data
            velocity_seg = [extract_velocity(ws_seg[i]) for i in range(len(ws_seg))]

            # Calculate features from each segment of keypoint data, joint angles and velocity
            feature_seg = [extract_feature(ws_seg[i], FS) for i in range(len(ws_seg))]
            feature_joint_angles_seg = [extract_feature(joint_angles_seg[i], FS) for i in
                                      range(len(joint_angles_seg))]
            feature_velocity_seg = [extract_feature(extract_velocity(ws_seg[i]), FS) for i in range(len(ws_seg))]

            # Concatenate all features
            feature_segs = np.concatenate([feature_seg, feature_joint_angles_seg, feature_velocity_seg], axis=1)

            all_feature.extend(feature_segs)
            all_label.extend([int(seg_label)]*len(ws_seg))

    df=pd.DataFrame(all_feature,columns=feature)
    df['label'] = all_label
    df.to_csv(f"./training_set_df/{USER_ID}.csv",index=False)
#     print(df)
    return df

In [7]:
df=make_csv("N01T1")
df

NameError: name 'KP_df' is not defined

In [9]:
IDS=os.listdir("./train")
for ID in IDS:
    print(ID)
    make_csv(ID)

N04T1_keypoint.csv


/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:5: RuntimeWarning: invalid value encountered in scalar divide
  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:73: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis=scipy.stats.kurtosis(data)
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:74: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew=scipy.stats.skew(data)
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:6: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)


N11T1_keypoint.csv


/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:5: RuntimeWarning: invalid value encountered in scalar divide
  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:73: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis=scipy.stats.kurtosis(data)
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:74: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew=scipy.stats.skew(data)


S07T2_keypoint.csv


...

/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:5: RuntimeWarning: invalid value encountered in scalar divide
  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:73: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis=scipy.stats.kurtosis(data)
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:74: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew=scipy.stats.skew(data)


S09T1_keypoint.csv


/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:5: RuntimeWarning: invalid value encountered in scalar divide
  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:73: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis=scipy.stats.kurtosis(data)
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:74: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew=scipy.stats.skew(data)
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:6: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)


S01T1_keypoint.csv


/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:5: RuntimeWarning: invalid value encountered in scalar divide
  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:73: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurtosis=scipy.stats.kurtosis(data)
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:74: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew=scipy.stats.skew(data)
/var/folders/qq/ckqvfttx4vs_68cbq08jts2c0000gn/T/ipykernel_50330/4229208218.py:6: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(cosine_angle)
